# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 3 2022 1:45PM,247768,19,N17791,"NAPP Technologies, LLC",Released
1,Oct 3 2022 1:40PM,247766,10,0086037167,ISDIN Corporation,Released
2,Oct 3 2022 1:40PM,247766,10,0086037171,ISDIN Corporation,Released
3,Oct 3 2022 1:40PM,247766,10,0086037170,ISDIN Corporation,Released
4,Oct 3 2022 1:40PM,247766,10,0086037174,ISDIN Corporation,Released
5,Oct 3 2022 1:40PM,247766,10,0086037175,ISDIN Corporation,Released
6,Oct 3 2022 1:40PM,247766,10,0086037173,ISDIN Corporation,Released
7,Oct 3 2022 1:40PM,247766,10,0086037172,ISDIN Corporation,Released
8,Oct 3 2022 1:40PM,247766,10,0086037181,ISDIN Corporation,Released
9,Oct 3 2022 1:40PM,247766,10,0086037176,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,247764,Released,12
35,247765,Released,31
36,247766,Released,39
37,247767,Released,8
38,247768,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247764,NaN,NaN,12.0
247765,NaN,NaN,31.0
247766,NaN,NaN,39.0
247767,NaN,NaN,8.0
247768,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247684,0.0,0.0,15.0
247692,0.0,0.0,1.0
247694,0.0,1.0,0.0
247706,0.0,0.0,1.0
247707,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247684,0,0,15
247692,0,0,1
247694,0,1,0
247706,0,0,1
247707,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247684,0,0,15
1,247692,0,0,1
2,247694,0,1,0
3,247706,0,0,1
4,247707,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247684,,,15
1,247692,,,1
2,247694,,1,
3,247706,,,1
4,247707,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 3 2022 1:45PM,247768,19,"NAPP Technologies, LLC"
1,Oct 3 2022 1:40PM,247766,10,ISDIN Corporation
40,Oct 3 2022 1:40PM,247767,10,ISDIN Corporation
48,Oct 3 2022 1:40PM,247765,10,ISDIN Corporation
79,Oct 3 2022 1:37PM,247764,10,ISDIN Corporation
91,Oct 3 2022 1:28PM,247762,15,"Person & Covey, Inc."
99,Oct 3 2022 1:22PM,247754,10,ISDIN Corporation
158,Oct 3 2022 1:22PM,247753,10,ISDIN Corporation
215,Oct 3 2022 1:21PM,247761,12,LF of America Corp.
216,Oct 3 2022 1:17PM,247694,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 3 2022 1:45PM,247768,19,"NAPP Technologies, LLC",,,1
1,Oct 3 2022 1:40PM,247766,10,ISDIN Corporation,,,39
2,Oct 3 2022 1:40PM,247767,10,ISDIN Corporation,,,8
3,Oct 3 2022 1:40PM,247765,10,ISDIN Corporation,,,31
4,Oct 3 2022 1:37PM,247764,10,ISDIN Corporation,,,12
5,Oct 3 2022 1:28PM,247762,15,"Person & Covey, Inc.",,,8
6,Oct 3 2022 1:22PM,247754,10,ISDIN Corporation,,,59
7,Oct 3 2022 1:22PM,247753,10,ISDIN Corporation,,,57
8,Oct 3 2022 1:21PM,247761,12,LF of America Corp.,,,1
9,Oct 3 2022 1:17PM,247694,21,"NBTY Global, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 1:45PM,247768,19,"NAPP Technologies, LLC",1,,
1,Oct 3 2022 1:40PM,247766,10,ISDIN Corporation,39,,
2,Oct 3 2022 1:40PM,247767,10,ISDIN Corporation,8,,
3,Oct 3 2022 1:40PM,247765,10,ISDIN Corporation,31,,
4,Oct 3 2022 1:37PM,247764,10,ISDIN Corporation,12,,
5,Oct 3 2022 1:28PM,247762,15,"Person & Covey, Inc.",8,,
6,Oct 3 2022 1:22PM,247754,10,ISDIN Corporation,59,,
7,Oct 3 2022 1:22PM,247753,10,ISDIN Corporation,57,,
8,Oct 3 2022 1:21PM,247761,12,LF of America Corp.,1,,
9,Oct 3 2022 1:17PM,247694,21,"NBTY Global, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 1:45PM,247768,19,"NAPP Technologies, LLC",1,,
1,Oct 3 2022 1:40PM,247766,10,ISDIN Corporation,39,,
2,Oct 3 2022 1:40PM,247767,10,ISDIN Corporation,8,,
3,Oct 3 2022 1:40PM,247765,10,ISDIN Corporation,31,,
4,Oct 3 2022 1:37PM,247764,10,ISDIN Corporation,12,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 1:45PM,247768,19,"NAPP Technologies, LLC",1.0,NaN,NaN
1,Oct 3 2022 1:40PM,247766,10,ISDIN Corporation,39.0,NaN,NaN
2,Oct 3 2022 1:40PM,247767,10,ISDIN Corporation,8.0,NaN,NaN
3,Oct 3 2022 1:40PM,247765,10,ISDIN Corporation,31.0,NaN,NaN
4,Oct 3 2022 1:37PM,247764,10,ISDIN Corporation,12.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 3 2022 1:45PM,247768,19,"NAPP Technologies, LLC",1.0,0.0,0.0
1,Oct 3 2022 1:40PM,247766,10,ISDIN Corporation,39.0,0.0,0.0
2,Oct 3 2022 1:40PM,247767,10,ISDIN Corporation,8.0,0.0,0.0
3,Oct 3 2022 1:40PM,247765,10,ISDIN Corporation,31.0,0.0,0.0
4,Oct 3 2022 1:37PM,247764,10,ISDIN Corporation,12.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4211545,304.0,8.0,1.0
12,247761,1.0,0.0,0.0
15,1238751,33.0,0.0,0.0
16,743135,11.0,0.0,0.0
19,1238774,23.0,32.0,0.0
20,247719,1.0,0.0,0.0
21,247694,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4211545,304.0,8.0,1.0
1,12,247761,1.0,0.0,0.0
2,15,1238751,33.0,0.0,0.0
3,16,743135,11.0,0.0,0.0
4,19,1238774,23.0,32.0,0.0
5,20,247719,1.0,0.0,0.0
6,21,247694,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,304.0,8.0,1.0
1,12,1.0,0.0,0.0
2,15,33.0,0.0,0.0
3,16,11.0,0.0,0.0
4,19,23.0,32.0,0.0
5,20,1.0,0.0,0.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,304.0
1,12,Released,1.0
2,15,Released,33.0
3,16,Released,11.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,8.0,0.0,0.0,0.0,32.0,0.0,1.0
Released,304.0,1.0,33.0,11.0,23.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,0.0,0.0,32.0,0.0,1.0
2,Released,304.0,1.0,33.0,11.0,23.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,0.0,0.0,32.0,0.0,1.0
2,Released,304.0,1.0,33.0,11.0,23.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()